In [1]:
import os
import data_processing_tool as dpt
from datetime import timedelta, date, datetime
# import args_parameter as args
import torch,torchvision
import numpy as np
import random

from torch.utils.data import Dataset,random_split
from torchvision import datasets, models, transforms

import time
import xarray as xr
from PIL import Image

import torch
import matplotlib as plt
import argparse
import sys
from torch.utils.data import DataLoader,random_split
from torchvision import datasets, models, transforms
import platform
from datetime import timedelta, date, datetime
from model import vdsr_dem
import torch.nn as nn
import numpy as np
import torch.optim as optim
import os
from math import log10
import time
# from PrepareData import ACCESS_BARRA_crps
import tqdm
from statsmodels.distributions.empirical_distribution import ECDF
# ===========================================================
# Training settings
# ===========================================================

station_50_code_lat_lon={'CARNAMAH': [128, 28],
 'MULLEWA': [138, 24],
 'WONGAN HILLS': [117, 35],
 'BADGINGARRA RESEARCH STN': [122, 24],
 'MOUNT BARKER': [83, 44],
 'ESPERANCE': [90, 82],
 'BENCUBBIN': [118, 46],
 'KELLERBERRIN': [110, 44],
 'BEVERLEY': [106, 37],
 'CORRIGIN': [104, 46],
 'HYDEN': [103, 55],
 'NARROGIN': [99, 39],
 'ONGERUP': [89, 51],
 'RAVENSTHORPE': [93, 65],
 'SALMON GUMS RES.STN.': [98, 80],
 'CEDUNA AMO': [106, 190],
 'CLEVE': [92, 215],
 'KYANCUTTA': [97, 206],
 'STREAKY BAY': [100, 194],
 'YONGALA': [98, 236],
 'PRICE': [86, 229],
 'WAROOKA': [80, 223],
 'ROSEDALE (TURRETFIELD RESEARCH CENTRE)': [84, 236],
 'MENINGIE': [73, 241],
 'KEITH': [70, 250],
 'SPRINGSURE COMET ST': [179, 320],
 'OAKEY AERO': [149, 354],
 'SURAT': [151, 329],
 'LAKE VICTORIA STORAGE': [88, 258],
 'COLLARENEBRI (ALBERT ST)': [129, 325],
 'BALRANALD (RSL)': [83, 279],
 'PEAK HILL POST OFFICE': [100, 321],
 'CONDOBOLIN AG RESEARCH STN': [97, 313],
 'NYNGAN AIRPORT': [111, 312],
 'TRANGIE RESEARCH STATION AWS': [107, 319],
 'MUNGINDI POST OFFICE': [134, 329],
 'QUIRINDI POST OFFICE': [111, 344],
 'ORANGE AGRICULTURAL INSTITUTE': [95, 329],
 'WAGGA WAGGA AMO': [78, 315],
 'GRENFELL (MANGANESE RD)': [90, 321],
 'COROWA AIRPORT': [71, 305],
 'NARRANDERA AIRPORT AWS': [82, 306],
 'HILLSTON AIRPORT': [93, 297],
 'LAKE CARGELLIGO AIRPORT': [95, 305],
 'MILDURA AIRPORT': [87, 266],
 'OUYEN (POST OFFICE)': [79, 268],
 'WARRACKNABEAL MUSEUM': [68, 269],
 'ECHUCA AERODROME': [69, 290],
 'KERANG': [73, 282],
 'ARARAT PRISON': [59, 274]}



class ACCESS_BARRA_crps(Dataset):
    '''

2.using my net to train one channel to one channel.
   
    '''
    def __init__(self,start_date=date(1990, 1, 1),end_date=date(1990,12 , 31),regin="AUS",lr_transform=None,hr_transform=None,shuffle=True,args=None):
#         print("=> BARRA_R & ACCESS_S1 loading")
#         print("=> from "+start_date.strftime("%Y/%m/%d")+" to "+end_date.strftime("%Y/%m/%d")+"")
        self.file_BARRA_dir = args.file_BARRA_dir
        self.file_ACCESS_dir = args.file_ACCESS_dir
        self.args=args
        
        self.lr_transform = lr_transform
        self.hr_transform = hr_transform

        self.start_date = start_date
        self.end_date = end_date
        
        self.regin = regin
        self.leading_time_we_use=args.leading_time_we_use

        self.ensemble_access=['e01','e02','e03','e04','e05','e06','e07','e08','e09','e10','e11']
        self.ensemble=[]
        for i in range(args.ensemble):
            self.ensemble.append(self.ensemble_access[i])
                
        self.dates = self.date_range(start_date, end_date)
        
        
        self.filename_list=self.get_filename_with_time_order(args.file_ACCESS_dir+"pr/daily/")
#         if not os.path.exists(args.file_ACCESS_dir+"pr/daily/"):
#             print(args.file_ACCESS_dir+"pr/daily/")
#             print("no file or no permission")
        
        
        _,_,date_for_BARRA,time_leading=self.filename_list[0]
        if shuffle:
            random.shuffle(self.filename_list)
        
        
#         if not os.path.exists("/g/data/ma05/BARRA_R/v1/forecast/spec/accum_prcp/1990/01/accum_prcp-fc-spec-PT1H-BARRA_R-v1-19900109T0600Z.sub.nc"):
#             print(self.file_BARRA_dir)
#             print("no file or no permission!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!")
        data_high=dpt.read_barra_data_fc_get_lat_lon(self.file_BARRA_dir,date_for_BARRA)
        self.lat=data_high[1]
        self.lon=data_high[1]
#         self.shape=(316, 376)


#         print(type(self.data_dem))
        
#             data_dem=dpt.add_lat_lon( dpt.read_dem(args.file_DEM_dir+"dem-9s1.tif"))
#             self.dem_data=dpt.interp_tensor_2d(dpt.map_aust_old(data_dem,xrarray=False) ,self.shape )
        

        
    def __len__(self):
        return len(self.filename_list)
    

    def date_range(self,start_date, end_date):
        """This function takes a start date and an end date as datetime date objects.
        It returns a list of dates for each date in order starting at the first date and ending with the last date"""
        return [start_date + timedelta(x) for x in range((end_date - start_date).days + 1)]

    
    def get_filename_with_no_time_order(self,rootdir):
        '''get filename first and generate label '''
        _files = []
        list = os.listdir(rootdir) #åˆ—å‡ºæ–‡ä»¶å¤¹ä¸‹æ‰€æœ‰çš„ç›®å½•ä¸Žæ–‡ä»¶
        for i in range(0,len(list)):
            path = os.path.join(rootdir,list[i])
            if os.path.isdir(path):
                _files.extend(self.get_filename_with_no_time_order(path))
            if os.path.isfile(path):
                if path[-3:]==".nc":
                    _files.append(path)
        return _files
    
    def get_filename_with_time_order(self,rootdir):
        '''get filename first and generate label ,one different w'''
        _files = []
        for date in self.dates:
            for i in range(self.leading_time_we_use,self.leading_time_we_use+1):
            

#                 filename="da_pr_"+date.strftime("%Y%m%d")+"_"+en+".nc"cd
#                 access_path=rootdir+en+"/"+"da_pr_"+date.strftime("%Y%m%d")+"_"+en+".nc"
#                 print(access_path)
                for en in self.ensemble:
                    access_path=rootdir+en+"/"+"da_pr_"+date.strftime("%Y%m%d")+"_"+en+".nc"
                    if os.path.exists(access_path):
                        
                    
                        if date==self.end_date and i==1:
                            break
                        path=[]
                        path.append(en)
                        barra_date=date+timedelta(i)
                        path.append(date)
                        path.append(barra_date)
                        path.append(i)
                        _files.append(path)

#     def get_filename_with_time_order(self,rootdir):
#         '''get filename first and generate label ,one different w'''
#         _files = []
#         for date in self.dates:
#             for en in self.ensemble:

# #                 filename="da_pr_"+date.strftime("%Y%m%d")+"_"+en+".nc"cd
#                 access_path=rootdir+en+"/"+"da_pr_"+date.strftime("%Y%m%d")+"_"+en+".nc"
# #                 print(access_path)
#                 if os.path.exists(access_path):
#                     for i in range(self.leading_time_we_use):
#                         if date==self.end_date and i==1:
#                             break
#                         path=[]
#                         path.append(en)
#                         barra_date=date+timedelta(i)
#                         path.append(date)
#                         path.append(barra_date)
#                         path.append(i)
#                         _files.append(path)                        

    #æœ€åŽåŽ»æŽ‰ç¬¬ä¸€è¡Œï¼Œç„¶åŽshuffle
        return _files

    def mapping(self,X,min_val=0.,max_val=255.):
        Xmin = np.min(X)
        Xmax = np.max(X)
        #å°†æ•°æ®æ˜ å°„åˆ°[-1,1]åŒºé—´ å³a=-1ï¼Œb=1
        a = min_val
        b = max_val
        Y = a + (b-a)/(Xmax-Xmin)*(X-Xmin)
        return Y
        
    def __getitem__(self, idx):
        '''
        from filename idx get id
        return lr,hr
        '''
        t=time.time()
#         data[var_name][:7,82:144,134:188]*86400
        #read_data filemame[idx]
        en,access_date,barra_date,time_leading=self.filename_list[idx]
        

#         lr=dpt.read_access_data(self.file_ACCESS_dir,en,access_date,time_leading,"pr")[82:144,134:188]*86400 #on hpc
        lr=dpt.read_access_data(self.file_ACCESS_dir,en,access_date,time_leading,"pr")*86400 #on win


#         lr=np.expand_dims(lr,axis=2)
        lr=dpt.interp_tensor_2d(lr,(79,94))
    
#         lr=np.expand_dims(self.mapping(lr),axis=2)
        label=dpt.read_SILO_data('../data/SILOstationData/nc/',barra_date)
        label=np.expand_dims(label,axis=1)
#         print(np.expand_dims(label,axis=1).shape)
        if self.args.zg:
            lr_zg=np.expand_dims(dpt.read_access_data(self.file_ACCESS_dir,en,access_date,time_leading,"zg"),axis=2)

        if self.args.psl:
            lr_psl=dpt.read_access_data(self.file_ACCESS_dir,en,access_date,time_leading,"psl")

        if self.args.tasmax:
            lr_tasmax=np.expand_dims(dpt.read_access_data(self.file_ACCESS_dir,en,access_date,time_leading,"tasmax"),axis=2)


        if self.args.tasmin:
            lr_tasmin=dpt.read_access_data(self.file_ACCESS_dir,en,access_date,time_leading,"tasmin")
            
#         if self.args.channels==1:
#             lr=np.repeat(lr,3,axis=2)
#         return self.lr_transform(Image.fromarray(lr)),self.lr_transform(Image.fromarray(self.data_dem)),self.hr_transform(Image.fromarray(label))

        return self.lr_transform(Image.fromarray(lr)),torch.tensor(int(en[1:])),np.array(label),torch.tensor(int(en[1:])),torch.tensor(int(access_date.strftime("%Y%m%d"))),torch.tensor(time_leading)

    

def crps(ensin,obs):
    '''
    @param ensin A vector of prediction
    @param obs  A vector of observations
    
'''

#     assert not np.isnan(ensin).any() and not np.isnan(obs).any(), "data contains nan"
         
    Fn = ECDF(ensin)
    xn=np.sort(np.unique(ensin))
    m=len(xn)
    dn=np.diff(xn)
    eq1=0
    eq2=0
    if(obs>xn[0] and obs<xn[m-1]): #obsåœ¨èŒƒå›´å†…
        k=np.max(np.where(xn<=obs))#å°äºŽobsçš„æœ€å¤§å€¼ä¸‹æ ‡
        x0 = xn[k] #å°äºŽobsçš„æœ€å¤§å€¼
        if k>0:
            eq1=np.sum(Fn(xn[0:k+1])**2*np.append(dn[0:k], obs - xn[k]))#å°äºŽobsçš„æ‰€æœ‰å€¼ çš„ ç™¾åˆ†æ¯”æ•° çš„å¹³æ–¹
        else:
            eq1 =np.sum(Fn(xn[0])**2*(obs - xn[0]))
        if k<m-2:

            eq2=np.sum((1-Fn(xn[k:m-1]))**2*np.append(xn[k+1] - obs, dn[(k+1):(m-1)]))
        else:
            eq2 =np.sum((1-Fn(xn[m-2]))**2*(xn[m-1] - obs))

    if obs <= xn[0]: # è§‚æµ‹å€¼åœ¨ä¹‹å¤–
        eq2 =np.sum(np.append(1, 1-Fn(xn[0:(m-1)]))**2*np.append(xn[0]-obs, dn))
    if obs >= xn[m-1]:
        eq1= np.sum(Fn(xn)**2*np.append(dn, obs - xn[m-1]))
            
    return eq1+eq2 




def vectcrps_v(fct_ens,obs):
    '''
    #' @param fct_ens A 2D prediction
    #' @param obs  A vector of observations
    #' @return a crps vector'''
    score =0

    
    fct_ens=fct_ens
    assert not np.isnan(fct_ens).any() and not np.isnan(obs).any(),"data contains nan"
    for i in range(obs.shape[0]):
#         print(fct_ens[:,i],obs[i])
        score+=crps(fct_ens[:,i],obs[i])
  
    return score


def vectcrps_m(fct_ens,obs):
    '''
    #' @param fct_ens A 2D prediction 11*1*1
    #' @param obs  A vector of observations
    #' @return a crps vector'''
    score =0
#     assert np.isnan(fct_ens).any() and np.isnan(obs).any(),"data contains nan"
    score_map=np.zeros((obs.shape[0],obs.shape[1]))
    for i in range(obs.shape[0]):
        for j in range(obs.shape[1]):
            score_map[i,j]=crps(fct_ens[:,i,j],obs[i,j])
#             score+=crps(fct_ens[:,i,j],obs[i,j])
    return score_map
    return score/(obs.shape[0]*obs.shape[1])   


def vectcrps_cali(fct_ens,obs):

    score =0
    mapp=np.load("mmap.npy")
#     assert np.isnan(fct_ens).any() and not np.isnan(obs).any(),"data contains nan"
    score_map=np.zeros((obs.shape[0],obs.shape[1]))
    for i in range(obs.shape[0]):
        for j in range(obs.shape[1]):
#             if (np.array(fct_ens[:,i,j],dtype=np.float32)>2000).any() or (mapp[i,j]==-1).any():
#             if (np.array(fct_ens[:,i,j],dtype=np.float32)>2000).any(): 
            if (np.array(fct_ens[:,mapp[i,j,0],mapp[i,j,1]],dtype=np.int)>=-0.00001).all():
                score_map[i,j]=crps(fct_ens[:,mapp[i,j,0],mapp[i,j,1]],obs[i,j])
            else:
                score_map[i,j]=np.inf
#             score+=crps(fct_ens[:,i,j],obs[i,j])
    return score_map

def vectcrps_m_50_SILO_station(fct_ens,obs):

    score =0
    assert not np.isnan(fct_ens).any() and not np.isnan(obs).any(),"data contains nan"
    score_map=np.zeros((50))
    for idx,i in enumerate(station_50_code_lat_lon.values()):
#         print(fct_ens[:,i[0],i[1]].shape,obs[idx].shape)
        score_map[idx]=crps(fct_ens[:,i[0],i[1]],obs[idx])
#     for i in range(obs.shape[0]):
#         for j in range(obs.shape[1]):
#             score_map[i,j]=crps(fct_ens[:,i,j],obs[i,j])
#             score+=crps(fct_ens[:,i,j],obs[i,j])
    return score_map

def write_log(log,args):
    print(log)
    if not os.path.exists("./save/"+args.train_name+"/"):
        os.mkdir("./save/"+args.train_name+"/")
    my_log_file=open("./save/"+args.train_name + '/train.txt', 'a')
#     log="Train for batch %d,data loading time cost %f s"%(batch,start-time.time())
    my_log_file.write(log + '\n')
    my_log_file.close()
    return



def main():
    parser = argparse.ArgumentParser(description='PyTorch Super Res Example')
    # Hardware specifications
    parser.add_argument('--n_threads', type=int, default=0,
                        help='number of threads for data loading')

    parser.add_argument('--cpu', action='store_true',help='cpu only?') 

    # hyper-parameters
    parser.add_argument('--train_name', type=str, default="cali_crps", help='training name')

    parser.add_argument('--batch_size', type=int, default=44, help='training batch size')
    parser.add_argument('--testBatchSize', type=int, default=4, help='testing batch size')
    parser.add_argument('--nEpochs', type=int, default=200, help='number of epochs to train for')
    parser.add_argument('--lr', type=float, default=0.0001, help='Learning Rate. Default=0.01')
    parser.add_argument('--seed', type=int, default=123, help='random seed to use. Default=123')

    # model configuration
    parser.add_argument('--upscale_factor', '-uf',  type=int, default=4, help="super resolution upscale factor")
    parser.add_argument('--model', '-m', type=str, default='vdsr', help='choose which model is going to use')

    #data
    parser.add_argument('--pr', type=bool, default=True,help='add-on pr?')

    parser.add_argument('--train_start_time', type=type(datetime(1990,1,25)), default=datetime(1990,1,2),help='r?')
    parser.add_argument('--train_end_time', type=type(datetime(1990,1,25)), default=datetime(1990,2,9),help='?')
    parser.add_argument('--test_start_time', type=type(datetime(2012,1,1)), default=datetime(2012,1,1),help='a?')
    parser.add_argument('--test_end_time', type=type(datetime(2012,12,31)), default=datetime(2012,12,31),help='')

    parser.add_argument('--dem', action='store_true',help='add-on dem?') 
    parser.add_argument('--psl', action='store_true',help='add-on psl?') 
    parser.add_argument('--zg', action='store_true',help='add-on zg?') 
    parser.add_argument('--tasmax', action='store_true',help='add-on tasmax?') 
    parser.add_argument('--tasmin', action='store_true',help='add-on tasmin?')
    parser.add_argument('--leading_time_we_use', type=int,default=1
                        ,help='add-on tasmin?')
    parser.add_argument('--ensemble', type=int, default=11,help='total ensambles is 11') 
    parser.add_argument('--channels', type=float, default=0,help='channel of data_input must') 
    #[111.85, 155.875, -44.35, -9.975]
    parser.add_argument('--domain', type=list, default=[112.9, 154.25, -43.7425, -9.0],help='dataset directory')

    parser.add_argument('--file_ACCESS_dir', type=str, default="../data/",help='dataset directory')

    parser.add_argument('--file_BARRA_dir', type=str, default="../data/barra_aus/",help='dataset directory')
    parser.add_argument('--file_DEM_dir', type=str, default="../DEM/",help='dataset directory')
    parser.add_argument('--precision', type=str, default='single',choices=('single', 'half','double'),help='FP precision for test (single | half)')

    args = parser.parse_args([])
    
    # def main():

#     init_date=date(1970, 1, 1)
#     start_date=date(1990, 1, 2)
#     end_date=date(2011,12,25)
    sys = platform.system()
    args.dem=False
    args.train_name="pr_dem"
    args.channels=0
    if args.pr:
        args.channels+=1
    if args.zg:
        args.channels+=1
    if args.psl:
        args.channels+=1
    if args.tasmax:
        args.channels+=1
    if args.tasmin:
        args.channels+=1
    if args.dem:
        args.channels+=1
    print("training statistics:")
    print("  ------------------------------")
    print("  trainning name  |  %s"%args.train_name)
    print("  ------------------------------")
    print("  num of channels | %5d"%args.channels)
    print("  ------------------------------")
    print("  num of threads  | %5d"%args.n_threads)
    print("  ------------------------------")
    print("  batch_size     | %5d"%args.batch_size)
    print("  ------------------------------")
    print("  using cpu only | %5d"%args.cpu)

    lr_transforms = transforms.Compose([
        transforms.Resize((316, 376)),
    #     transforms.RandomResizedCrop(IMG_SIZE),
    #     transforms.RandomHorizontalFlip(),
    #     transforms.RandomRotation(30),
        transforms.ToTensor()
    #     transforms.Normalize(IMG_MEAN, IMG_STD)
    ])

    hr_transforms = transforms.Compose([
    #         transforms.Resize((316, 376)),
    #     transforms.RandomResizedCrop(IMG_SIZE),
    #     transforms.RandomHorizontalFlip(),
    #     transforms.RandomRotation(30),
#         transforms.ToTensor()
    #     transforms.Normalize(IMG_MEAN, IMG_STD)
    ])

    data_set=ACCESS_BARRA_crps(args.test_start_time,args.test_end_time,lr_transform=lr_transforms,hr_transform=hr_transforms,shuffle=False,args=args)



    #     #######################################################################

    test_data=DataLoader(data_set,
                                            batch_size=args.batch_size,
                                            shuffle=False,
                                num_workers=args.n_threads,drop_last=True)

    #     #######################################################################


    device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
    net=torch.load("./save/vdsr_pr/best_test.pth",map_location=device)
    # net=torch.load("../data/model/vdsr_pr/best_test.pth")


    def prepare( l, device=False):
        def _prepare(tensor):
            if args.precision == 'half': tensor = tensor.half()
            if args.precision == 'single': tensor = tensor.float()
            return tensor.to(device)

        return [_prepare(_l) for _l in l]

    #     ##############################################
    write_log("start",args)
    #     max_error=np.inf
    #     val_max_error=np.inf

    #     print(data_set.filename_list)

    # for e in range(args.nEpochs):
    #         loss=0
    for lead in range(30):
        args.leading_time_we_use=lead

        data_set=ACCESS_BARRA_crps(args.test_start_time,args.test_end_time,lr_transform=lr_transforms,hr_transform=hr_transforms,shuffle=False,args=args)


        test_data=DataLoader(data_set,
                                                batch_size=args.batch_size,
                                                shuffle=False,
                                    num_workers=args.n_threads,drop_last=False)


        crps_score_vsdr=[]
        result_vdsr=[]
        start=time.time()
        fmt = '%Y%m%d'


    #         test_data=tqdm.tqdm(test_data)
        for batch, (pr,dem,hr,en,data_time,idx) in enumerate(test_data):

            pr,dem,hr= prepare([pr,dem,hr],device)
        #             print(en,data_time,idx)
        #             optimizer.zero_grad()
            with torch.set_grad_enabled(False):

                sr = net(pr)
                sr_np=sr.cpu().numpy()
                hr_np=hr.cpu().numpy()
#                 print(sr_np.shape)
#                 print(hr_np.shape)                
                for i in range(args.batch_size//args.ensemble):
                    a=np.squeeze( sr_np[i*args.ensemble:(i+1)*args.ensemble])
                    b=np.squeeze(hr_np[i*args.ensemble])
#                     print(a.shape,b.shape)
        #             print(a.shape,b.shape)
        #             skil=vectcrps_m(a,b)
                    skil=vectcrps_m_50_SILO_station(a,b)
    #                 print(skil.shape)
    #                 time_tuple = time.strptime(str(data_time[i*args.ensemble].item()), fmt)
    #                 year, month, day = time_tuple[:3]
    #                 a_date = date(year, month, day)
    #                 print(data_time,idx)
    #                 np.save("../crps/vsdr_pr/"+(a_date+timedelta(idx[i*args.ensemble].item())).strftime("%Y%m%d")+'_50station',skil)


                    crps_score_vsdr.append(skil)
                    result_vdsr.append(a)
        if not os.path.exists("./save/crps/my_50_station/"):
            os.makedirs("./save/crps/my_50_station/")
        np.save("./save/crps/my_50_station/lead_time"+str(lead),crps_score_vsdr)
        if not os.path.exists("./save/crps/my_30_result/"):
            os.makedirs("./save/crps/my_30_result/")
        np.save("./save/crps/my_30_result/lead_time"+str(lead),result_vdsr)
        print(str(lead)+" : "+str(np.array(crps_score_vsdr).mean()),",")



In [2]:
main()    


training statistics:
  ------------------------------
  trainning name  |  pr_dem
  ------------------------------
  num of channels |     1
  ------------------------------
  num of threads  |     0
  ------------------------------
  batch_size     |    44
  ------------------------------
  using cpu only |     0
start
0 : 1.486367747044959 ,
1 : 1.208022220511829 ,
2 : 1.2600500447530723 ,
3 : 1.1601097633008275 ,
4 : 1.2541658485812548 ,
5 : 1.1334894911725082 ,
6 : 0.8260255117304511 ,
7 : 0.9757547740401632 ,
8 : 1.2655047462526947 ,
9 : 1.0645102727475952 ,
10 : 0.9990847977592171 ,
11 : 0.8474732891807892 ,
12 : 1.17049723389071 ,
13 : 1.243240215275243 ,
14 : 0.8369664293557435 ,
15 : 1.0315515906891521 ,
16 : 1.5793241649317773 ,
17 : 1.0826280511899589 ,
18 : 0.9076717461557803 ,
19 : 1.0126065697702493 ,
20 : 1.1724180761977963 ,
21 : 1.2559583619864596 ,
22 : 0.8340822778011104 ,
23 : 1.1263395503417497 ,
24 : 1.4952074247160618 ,
25 : 1.0113097396650859 ,
26 : 0.8847910455

In [31]:
vectcrps_m_50_SILO_station

def vectcrps_m_50_SILO_station(fct_ens,obs):

    score =0
    assert not np.isnan(fct_ens).any() and not np.isnan(obs).any(),"data contains nan"
    score_map=np.zeros((50))
    for idx,i in enumerate(station_50_index.values()):
        score_map[idx]=crps(fct_ens[:,i[0],i[1]],obs[idx])
#     for i in range(obs.shape[0]):
#         for j in range(obs.shape[1]):
#             score_map[i,j]=crps(fct_ens[:,i,j],obs[i,j])
#             score+=crps(fct_ens[:,i,j],obs[i,j])
    return score_map

In [10]:
parser = argparse.ArgumentParser(description='PyTorch Super Res Example')
# Hardware specifications
parser.add_argument('--n_threads', type=int, default=0,
                    help='number of threads for data loading')

parser.add_argument('--cpu', action='store_true',help='cpu only?') 

# hyper-parameters
parser.add_argument('--train_name', type=str, default="cali_crps", help='training name')

parser.add_argument('--batch_size', type=int, default=44, help='training batch size')
parser.add_argument('--testBatchSize', type=int, default=4, help='testing batch size')
parser.add_argument('--nEpochs', type=int, default=200, help='number of epochs to train for')
parser.add_argument('--lr', type=float, default=0.0001, help='Learning Rate. Default=0.01')
parser.add_argument('--seed', type=int, default=123, help='random seed to use. Default=123')

# model configuration
parser.add_argument('--upscale_factor', '-uf',  type=int, default=4, help="super resolution upscale factor")
parser.add_argument('--model', '-m', type=str, default='vdsr', help='choose which model is going to use')

#data
parser.add_argument('--pr', type=bool, default=True,help='add-on pr?')

parser.add_argument('--train_start_time', type=type(datetime(1990,1,25)), default=datetime(1990,1,2),help='r?')
parser.add_argument('--train_end_time', type=type(datetime(1990,1,25)), default=datetime(1990,2,9),help='?')
parser.add_argument('--test_start_time', type=type(datetime(2012,1,1)), default=datetime(2012,1,1),help='a?')
parser.add_argument('--test_end_time', type=type(datetime(2012,12,31)), default=datetime(2012,12,31),help='')

parser.add_argument('--dem', action='store_true',help='add-on dem?') 
parser.add_argument('--psl', action='store_true',help='add-on psl?') 
parser.add_argument('--zg', action='store_true',help='add-on zg?') 
parser.add_argument('--tasmax', action='store_true',help='add-on tasmax?') 
parser.add_argument('--tasmin', action='store_true',help='add-on tasmin?')
parser.add_argument('--leading_time_we_use', type=int,default=1
                    ,help='add-on tasmin?')
parser.add_argument('--ensemble', type=int, default=11,help='total ensambles is 11') 
parser.add_argument('--channels', type=float, default=0,help='channel of data_input must') 
#[111.85, 155.875, -44.35, -9.975]
parser.add_argument('--domain', type=list, default=[112.9, 154.25, -43.7425, -9.0],help='dataset directory')

parser.add_argument('--file_ACCESS_dir', type=str, default="../data/",help='dataset directory')

parser.add_argument('--file_BARRA_dir', type=str, default="../data/barra_aus/",help='dataset directory')
parser.add_argument('--file_DEM_dir', type=str, default="../DEM/",help='dataset directory')
parser.add_argument('--precision', type=str, default='single',choices=('single', 'half','double'),help='FP precision for test (single | half)')

args = parser.parse_args([])

# def main():

#     init_date=date(1970, 1, 1)
#     start_date=date(1990, 1, 2)
#     end_date=date(2011,12,25)
sys = platform.system()
args.dem=False
args.train_name="pr_dem"
args.channels=0
if args.pr:
    args.channels+=1
if args.zg:
    args.channels+=1
if args.psl:
    args.channels+=1
if args.tasmax:
    args.channels+=1
if args.tasmin:
    args.channels+=1
if args.dem:
    args.channels+=1
print("training statistics:")
print("  ------------------------------")
print("  trainning name  |  %s"%args.train_name)
print("  ------------------------------")
print("  num of channels | %5d"%args.channels)
print("  ------------------------------")
print("  num of threads  | %5d"%args.n_threads)
print("  ------------------------------")
print("  batch_size     | %5d"%args.batch_size)
print("  ------------------------------")
print("  using cpu only | %5d"%args.cpu)

lr_transforms = transforms.Compose([
    transforms.Resize((316, 376)),
#     transforms.RandomResizedCrop(IMG_SIZE),
#     transforms.RandomHorizontalFlip(),
#     transforms.RandomRotation(30),
    transforms.ToTensor()
#     transforms.Normalize(IMG_MEAN, IMG_STD)
])

hr_transforms = transforms.Compose([
#         transforms.Resize((316, 376)),
#     transforms.RandomResizedCrop(IMG_SIZE),
#     transforms.RandomHorizontalFlip(),
#     transforms.RandomRotation(30),
#         transforms.ToTensor()
#     transforms.Normalize(IMG_MEAN, IMG_STD)
])

data_set=ACCESS_BARRA_crps(args.test_start_time,args.test_end_time,lr_transform=lr_transforms,hr_transform=hr_transforms,shuffle=False,args=args)


training statistics:
  ------------------------------
  trainning name  |  pr_dem
  ------------------------------
  num of channels |     1
  ------------------------------
  num of threads  |     0
  ------------------------------
  batch_size     |    44
  ------------------------------
  using cpu only |     0


In [12]:
data_set.filename_list

[['e01',
  datetime.datetime(2012, 1, 1, 0, 0),
  datetime.datetime(2012, 1, 2, 0, 0),
  1],
 ['e02',
  datetime.datetime(2012, 1, 1, 0, 0),
  datetime.datetime(2012, 1, 2, 0, 0),
  1],
 ['e03',
  datetime.datetime(2012, 1, 1, 0, 0),
  datetime.datetime(2012, 1, 2, 0, 0),
  1],
 ['e04',
  datetime.datetime(2012, 1, 1, 0, 0),
  datetime.datetime(2012, 1, 2, 0, 0),
  1],
 ['e05',
  datetime.datetime(2012, 1, 1, 0, 0),
  datetime.datetime(2012, 1, 2, 0, 0),
  1],
 ['e06',
  datetime.datetime(2012, 1, 1, 0, 0),
  datetime.datetime(2012, 1, 2, 0, 0),
  1],
 ['e07',
  datetime.datetime(2012, 1, 1, 0, 0),
  datetime.datetime(2012, 1, 2, 0, 0),
  1],
 ['e08',
  datetime.datetime(2012, 1, 1, 0, 0),
  datetime.datetime(2012, 1, 2, 0, 0),
  1],
 ['e09',
  datetime.datetime(2012, 1, 1, 0, 0),
  datetime.datetime(2012, 1, 2, 0, 0),
  1],
 ['e10',
  datetime.datetime(2012, 1, 1, 0, 0),
  datetime.datetime(2012, 1, 2, 0, 0),
  1],
 ['e11',
  datetime.datetime(2012, 1, 1, 0, 0),
  datetime.datetime(20